In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable

import numpy as np
import cvxpy as cp

from free_flyer.free_flyer import FreeFlyer
import pickle, os

In [2]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
x_test = test_data[1]
c_test = test_data[4]
times_test = test_data[5]
test_file.close()
n_test = x_test.shape[0]

# image encodings

In [3]:
from solvers.mlopt_ff import MLOPT_FF

In [4]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

mlopt_ts_obj = MLOPT_FF(system, prob, prob_features)

n_features = 4
mlopt_ts_obj.construct_strategies(n_features, train_data)
print(mlopt_ts_obj.n_strategies)

430


In [7]:
mlopt_ts_obj.setup_network()

fn_saved = 'models/mloptff_free_flyer_20200706_0823.pt'
mlopt_ts_obj.load_network(fn_saved)

mlopt_ts_obj.model

Loading presaved classifier model from models/mloptff_free_flyer_20200706_0823.pt


CNNet(
  (conv_activation): ReLU()
  (ff_activation): ReLU()
  (conv_layers): ModuleList(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(2, 2))
    (1): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
    (2): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
  )
  (ff_layers): ModuleList(
    (0): Linear(in_features=260, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=430, bias=True)
  )
)

In [6]:
mlopt_ts_obj.training_params['TRAINING_ITERATIONS'] = 500
mlopt_ts_obj.train(train_data=train_data, verbose=False)

KeyboardInterrupt: 

In [8]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_success, cost, total_time, n_evals = mlopt_ts_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

59.034816145902205
